In [1]:
import boto3
import requests
import time

In [2]:
# Personal Info:

dolby_api = '[YOUR DOLBY API KEY HERE]'

aws_key = '[YOUR AWS KEY HERE]'
aws_secret = '[YOUR AWS SECRET KEY HERE]'


In [3]:
# Instantiate Boto Client

s3_client = boto3.client('s3',
                         aws_access_key_id = aws_key,
                         aws_secret_access_key = aws_secret)

In [8]:
files = s3_client.list_objects_v2(Bucket = 'project-5-audio-files') # confirm we can nagivate to bucket


# files
for file in files['Contents']:
    if file['Key'].endswith('.mp3') and file['Key'].startswith('raw/'):
        print(file['Key'])

raw/25818-20200502-1222.mp3
raw/25818-20200502-1332.mp3
raw/25818-20200502-1401.mp3
raw/25818-20200502-1431.mp3
raw/25818-20200502-1501.mp3
raw/25818-20200502-1531.mp3
raw/25818-20200502-1601.mp3
raw/25818-20200502-1631.mp3
raw/25818-20200502-1700.mp3
raw/25818-20200502-1730.mp3
raw/25818-20200502-1800.mp3
raw/25818-20200502-1830.mp3
raw/25818-20200502-1900.mp3
raw/25818-20200502-1930.mp3
raw/25818-20200502-2059.mp3
raw/25818-20200502-2129.mp3
raw/25818-20200502-2159.mp3
raw/25818-20200502-2229.mp3
raw/25818-20200502-2258.mp3
raw/25818-20200502-2328.mp3
raw/25818-20200502-2358.mp3


In [5]:
# Generate presigned urls

aws_input_url = s3_client.generate_presigned_url('get_object',
                                                 Params = {'Bucket': 'project-5-audio-files',
                                                           'Key': '25818-20200501-1210.mp3'})

aws_output_url = s3_client.generate_presigned_url('put_object',
                                                  Params = {'Bucket': 'project-5-audio-files',
                                                            'Key': 'enhanced/enhanced-25818-20200501-1210.mp3'})

In [45]:
# POST to enhance (should send file to dolby, enhance, and write new file to specified output location)
post_request_enhance = requests.post('https://api.dolby.com/media/enhance',
                                     headers = {'x-api-key': dolby_api},
                                     json = {
                                         "input": aws_input_url,
                                         "output": aws_output_url,
                                         "audio": { 
                                             "loudness": {
                                                 "enable": True
                                             },
                                             "dynamics": {
                                                 "range_control": {
                                                     "enable": True,
                                                     "amount": "max"
                                                 }
                                             },
                                             "noise": {
                                                 "reduction": {
                                                     "enable": False,
                                                     "amount": "max"
                                                 }
                                             },
                                             "filter": {
                                                 "high_pass": {
                                                     "enable": True,
                                                     "frequency": 80
                                                 }
                                             },
                                             "speech": {
                                                 "isolation": {
                                                     "enable": True,
                                                     "amount": 100},
                                                 "sibilance": {
                                                     "reduction": {
                                                         "enable": True,
                                                         "amount": "low"}
                                                 } # wraps sibilance
                                             } # wraps speech
                                         } #wraps audio
                                     })

In [2]:
# post_request_enhance.json()

In [4]:
# Get Job ID
job_id = post_request_enhance.json()['job_id']

# Create URL to check status
status_url = 'https://api.dolby.com/media/enhance?job_id=' + job_id

# Request status
status = requests.get(status_url, 
                      headers = {'x-api-key': dolby_api})

status = status.json()

time_start = time.time()

# Keep checking status
while status['status'] == 'Running':
    status = requests.get(status_url, 
                          headers = {'x-api-key': dolby_api})
    print(status.json())
    print(f'Time elapsed: {time.time() - time_start}s')
    status = status.json()
    time.sleep(30)